# Diplomatura en Ciencia de Datos    FAMAF

## Análisis de Texto

En este ejercicio se realizó el análisis de un texto de The Project Gutemberg usando spacy y textacy.

Al principio importamos las librerías, creamos una función para leer el texto y cargamos el modelo de spacy para procesar el texto en español.

In [35]:
# <!-- collapse=True -->
# Importando las librerías que vamos a utilizar
import matplotlib.pyplot as plt 
import textacy
import spacy
from textacy.datasets import Wikipedia
from collections import Counter, defaultdict
import warnings; warnings.simplefilter('ignore')

# graficos incrustados
%matplotlib inline

# función auxiliar
def leer_texto(texto):
    """Funcion auxiliar para leer un archivo de texto"""
    with open(texto, 'r') as text:
        return text.read()
    
# Cargando el modelo en español de spacy
nlp = spacy.load('es_core_news_md')

Ahora cargamos el texto Los Puritanos extraído del proyecto Gutemberg. El encabezado fue limpiado a mano.

In [36]:
# traemos el texto y lo metemos al dataset

texto = leer_texto('../Downloads/los_puritanos.txt')
texto_procesado = nlp(texto)


Contamos las palabras y limpiamos los signos de puntuación y stop words.

In [37]:
words = [token.text for token in texto_procesado if token.is_stop != True and token.is_punct != True and token.is_alpha == True and len(token.text) > 2]
word_freq = Counter(words)
common_words = word_freq.most_common()


Imprimimos la frequencia de las palabras

In [38]:
common_words

[('casa', 52),
 ('the', 33),
 ('calle', 31),
 ('Juan', 25),
 ('iba', 25),
 ('voz', 22),
 ('instante', 21),
 ('niña', 21),
 ('ciego', 20),
 ('cabeza', 19),
 ('Santiago', 18),
 ('Madrid', 18),
 ('noche', 17),
 ('ojos', 17),
 ('hermano', 16),
 ('Luisa', 16),
 ('corazón', 15),
 ('cosa', 15),
 ('Teresa', 15),
 ('Dios', 14),
 ('pasar', 14),
 ('vista', 14),
 ('balcón', 14),
 ('Asunción', 14),
 ('padre', 13),
 ('mano', 13),
 ('volvió', 13),
 ('vergüenza', 13),
 ('brazo', 13),
 ('calles', 12),
 ('vida', 11),
 ('pobre', 11),
 ('inmediatamente', 11),
 ('papá', 11),
 ('mamá', 11),
 ('hacía', 10),
 ('podía', 10),
 ('sorpresa', 10),
 ('cantar', 10),
 ('nieve', 10),
 ('efecto', 10),
 ('dije', 10),
 ('puerta', 10),
 ('cuarto', 9),
 ('sol', 9),
 ('cabo', 9),
 ('cielo', 9),
 ('gente', 9),
 ('muerte', 9),
 ('manos', 9),
 ('that', 9),
 ('paseo', 9),
 ('Lolita', 9),
 ('muñeca', 9),
 ('único', 8),
 ('años', 8),
 ('hombre', 8),
 ('salir', 8),
 ('mujer', 8),
 ('vuelta', 8),
 ('punto', 8),
 ('seguía', 8),
 ('p

A continuación filtramos los personajes contenidos en los cuentos.  En este ejercicio tuvimos muchos problemas y no pudimos resolverlo solos. Finalmente con ayuda de nuestros compañeros pudimos empezar a terminarlo. A través de la librería spacy la función texto_procesado.ents nos ayudó a devolver las diferentes entidades en el texto. Estas entidades pueden ser conjuntos de palabras y son importantes para identificar los nombres completos (en caso de haber un nombre y apellido). Accediendo al label de cada entidad contenida en nuestro texto podremos saber si se trata o no de un personaje. Pero nos encontramos con falsos positivos en los resultados y debimos aplicar ciertos procedimientos para filtrarlos. Observando los resultados encontramos diversa cantidad de símbolos a los costados de nuestras entidades, así como el string '-PRON-' el cual no aportaba nada, con lo cual se decidió filtrar todos estos símbolos de nuestros strings. Además se quitaron los saltos de línea de las entidades y se eliminaron las que estaban vacías. Finalmente descartamos las entidades que tuvieran 2 caracteres o menos (nos ayudaba a filtrar muchas palabras basura) y los nombres de personajes en general cuentan con mas caracteres. Por último notamos que teníamos muchos verbos que eran considerados personajes, para solucionar esto volvimos a pasar el texto por nuestro modelo de spacy y recorrimos cada personaje revisando si contenía verbos, en caso de tenerlos, los descartabamos. 

In [40]:
personajes = []
for ent in texto_procesado.ents:
    if ent.label_ == 'PER' and ent.lemma_ != '' and ent.lemma_.replace('-PRON-','').strip() != '' and len(ent.lemma_.replace('\n','').strip().strip('.').strip('_').strip('¡').strip('!').strip('¿').strip('?'))>2:
        substring = texto[ent.start_char: ent.end_char].replace('\n','').strip().strip('.').strip('_').strip('¡').strip('!').strip('¿').strip('?')
        if len(substring.strip()) > 2:
            personajes.append(substring)

personajes_filtrados = []
for personaje in personajes:
    personaje_model = nlp(personaje)
    is_verb = False
    for token in personaje_model:
        if token.pos_ == 'VERB':
            is_verb = True
    if not is_verb:
        personajes_filtrados.append(personaje)
pers_frec = Counter(personajes_filtrados)

Volviendo al tema de los datos infiltrados o falsos positivos, los mismos se dan debido a que el modelo que reconoce los personajes no está entrenado con un cierto dateset, este dataset no es perfecto y tiene una determinada precisón. Si observamos los falsos positivos, éstos comienzan con mayúscula, muchas veces están seguidos de verbos, están al comienzo de la oración o como sujeto, todas características compartidas con los personajes. Todo esto hace que el modelo pueda cometer errores y clasificar mal las entidades. 

In [42]:
pers_frec

Counter({'Alcela': 1,
         'Alcé': 1,
         'Asunción': 11,
         'Así': 1,
         'Barquillo': 1,
         'Beethoven': 1,
         'Bellini': 2,
         'Calle': 1,
         'Chonchita': 2,
         'Concha': 1,
         'Cuidado': 1,
         'Cybele': 1,
         'D. Ramón': 6,
         'Despedime': 1,
         'Dios': 3,
         'Dios?[76': 1,
         'Elvira': 1,
         'Enjugad': 1,
         'Envolví': 1,
         'Fuencarral': 1,
         'Has': 1,
         'Infantas': 4,
         'Jiménez': 1,
         'Juan': 17,
         'Juanillo': 4,
         'Lola': 5,
         'Los Puritanos': 1,
         'Luisa': 14,
         'Lástima': 1,
         'Madre Santísima': 1,
         'Manuel': 2,
         'Paco': 1,
         'Paco Núñez': 1,
         'Pepito': 5,
         'Perdón': 1,
         'Quién': 1,
         'Rey': 1,
         'Rodeáronlas': 1,
         'Santiago': 14,
         'Santo Domingo': 1,
         'Satisfecho': 1,
         'Será': 2,
         'Soy': 1,
       

Falta realizar el análisis propio sobre el texto.